# PEFT - Quicktour

Esse guia mostra os ponto principais usados para se rodar PEFT usando o google Colab

Link do guia base: https://huggingface.co/docs/peft/quicktour


## Treino

Cada método PEFT é definido por uma classe peftConfig que armazena todos os parametros importantes para construir o modelo PeftModel. Por exemplo, para treinar usando LoRA, carregue e crie uma classe LoraConfig e especifique os seguintes parametros:

* task_type: A tarefa para treinamento (*sequence-to-sequence language* modeling nesse caso)
* inference_mode: Se o modelo será usado para inferencia ou não
* r: A dimensão das low-rank matrices
* lora_alpha: Fator de escala para as low-rank matrices
* lora_dropout: Valor de dropout das camadas LoRA


In [12]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                         inference_mode=False,
                         r=8,
                         lora_alpha=32,
                         lora_dropout=0.1)

In [11]:
TaskType

<enum 'TaskType'>

PS: Veja o arquivo de [LoraConfig](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/lora#peft.LoraConfig) para mais detalhes

In [9]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [13]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
"output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282"

trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705


'output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282'

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="danielhsf/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

NameError: name 'Trainer' is not defined